# ICEEMDAN-MKL: Real Financial Data Demo

This notebook demonstrates ICEEMDAN decomposition on real market data using yfinance.

**Use cases covered:**
- Denoising price series
- Trend extraction
- SNR analysis (is this market tradeable?)
- Multi-asset comparison

In [ ]:
# Install dependencies if needed
# !pip install yfinance matplotlib numpy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta

# Import our ICEEMDAN
from iceemdan import ICEEMDAN, ProcessingMode

print(f"ICEEMDAN loaded successfully")

# Plot settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11

## 1. Load Real Market Data

In [ ]:
# Download data
symbols = ['BTC-USD', 'ETH-USD', 'SPY', 'AAPL', 'TSLA']
end_date = datetime.now()
start_date = end_date - timedelta(days=365)

data = {}
for sym in symbols:
    ticker = yf.Ticker(sym)
    df = ticker.history(start=start_date, end=end_date, interval='1d')
    if len(df) > 100:
        data[sym] = df['Close'].values
        print(f"{sym}: {len(data[sym])} samples")
    else:
        print(f"{sym}: Not enough data")

## 2. Basic Decomposition

In [ ]:
# Create ICEEMDAN decomposer with Finance mode
decomposer = ICEEMDAN(mode=ProcessingMode.FINANCE)
decomposer.ensemble_size = 100
decomposer.noise_std = 0.2

# Decompose BTC
symbol = 'BTC-USD'
prices = data[symbol]

print(f"Decomposing {symbol} ({len(prices)} samples)...")
imfs, residue, diag = decomposer.decompose(prices, diagnostics=True)

print(f"IMFs extracted: {len(imfs)}")
print(f"Orthogonality index: {diag['orthogonality_index']:.4f}")
print(f"Valid: {diag['valid']}")

In [ ]:
# Plot decomposition
n_imfs = len(imfs)
fig, axes = plt.subplots(n_imfs + 2, 1, figsize=(14, 2.5 * (n_imfs + 2)))

# Original signal
axes[0].plot(prices, 'b-', linewidth=0.8)
axes[0].set_ylabel('Price')
axes[0].set_title(f'{symbol} - ICEEMDAN Decomposition')

# IMFs
for i, imf in enumerate(imfs):
    axes[i + 1].plot(imf, 'b-', linewidth=0.5)
    axes[i + 1].set_ylabel(f'IMF {i}')

# Residue
axes[-1].plot(residue, 'r-', linewidth=1.0)
axes[-1].set_ylabel('Residue')
axes[-1].set_xlabel('Sample Index')

plt.tight_layout()
plt.show()

## 3. Denoising Application

In [ ]:
# Denoise: remove first 2 high-frequency IMFs
noise_imfs = 2
denoised = imfs[noise_imfs:].sum(axis=0) + residue
noise = imfs[:noise_imfs].sum(axis=0)

fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# Original vs Denoised
axes[0].plot(prices, 'b-', alpha=0.7, linewidth=0.8, label='Original')
axes[0].plot(denoised, 'r-', linewidth=1.2, label='Denoised')
axes[0].set_ylabel('Price')
axes[0].set_title(f'{symbol} - Denoising (removed IMF 0-{noise_imfs-1})')
axes[0].legend()

# Detrended (price - residue)
detrended = prices - residue
axes[1].plot(detrended, 'g-', linewidth=0.8)
axes[1].axhline(y=0, color='k', linestyle='--', alpha=0.3)
axes[1].set_ylabel('Detrended')
axes[1].set_title('Detrended Price (trend removed)')

# Extracted noise
axes[2].plot(noise, 'm-', linewidth=0.5)
axes[2].axhline(y=0, color='k', linestyle='--', alpha=0.3)
axes[2].set_ylabel('Noise')
axes[2].set_xlabel('Sample Index')
axes[2].set_title('Extracted Noise (high-freq IMFs)')

plt.tight_layout()
plt.show()

## 4. SNR Analysis - Is This Market Tradeable?

In [ ]:
def compute_snr(prices, decomposer, noise_imfs=2):
    """Compute Signal-to-Noise Ratio from ICEEMDAN decomposition."""
    imfs, residue = decomposer.decompose(prices)
    
    # Noise = energy in first N IMFs
    noise_energy = sum(np.sum(imfs[i]**2) for i in range(min(noise_imfs, len(imfs))))
    
    # Signal = energy in remaining IMFs + residue
    signal_energy = sum(np.sum(imfs[i]**2) for i in range(noise_imfs, len(imfs)))
    signal_energy += np.sum(residue**2)
    
    snr = signal_energy / noise_energy if noise_energy > 0 else float('inf')
    noise_fraction = noise_energy / (signal_energy + noise_energy)
    
    return {
        'snr': snr,
        'noise_fraction': noise_fraction,
        'n_imfs': len(imfs)
    }

In [ ]:
# Compare SNR across assets
print("SNR Analysis - Is this market tradeable?")
print("=" * 50)
print(f"{'Asset':<12} {'SNR':>10} {'Noise %':>10} {'IMFs':>6} {'Verdict':<15}")
print("-" * 50)

snr_results = {}
for sym, prices in data.items():
    result = compute_snr(prices, decomposer)
    snr_results[sym] = result
    
    # Verdict based on SNR
    if result['snr'] > 10:
        verdict = "Strong signal"
    elif result['snr'] > 3:
        verdict = "Tradeable"
    elif result['snr'] > 1:
        verdict = "Weak signal"
    else:
        verdict = "Noise - skip"
    
    print(f"{sym:<12} {result['snr']:>10.2f} {result['noise_fraction']*100:>9.1f}% {result['n_imfs']:>6} {verdict:<15}")

In [ ]:
# Visualize SNR comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

symbols = list(snr_results.keys())
snrs = [snr_results[s]['snr'] for s in symbols]
noise_fracs = [snr_results[s]['noise_fraction'] * 100 for s in symbols]

# SNR bar chart
colors = ['green' if s > 3 else 'orange' if s > 1 else 'red' for s in snrs]
axes[0].bar(symbols, snrs, color=colors, alpha=0.7, edgecolor='black')
axes[0].axhline(y=3, color='orange', linestyle='--', label='Tradeable threshold')
axes[0].axhline(y=1, color='red', linestyle='--', label='Noise threshold')
axes[0].set_ylabel('Signal-to-Noise Ratio')
axes[0].set_title('SNR by Asset')
axes[0].legend()

# Noise fraction bar chart
axes[1].bar(symbols, noise_fracs, color='purple', alpha=0.7, edgecolor='black')
axes[1].set_ylabel('Noise Fraction (%)')
axes[1].set_title('High-Frequency Noise Content')

plt.tight_layout()
plt.show()

## 5. IMF Energy Distribution

In [ ]:
def plot_energy_distribution(symbol, prices, decomposer):
    """Plot energy distribution across IMFs."""
    imfs, residue = decomposer.decompose(prices)
    
    # Compute energies
    energies = [np.sum(imf**2) for imf in imfs]
    energies.append(np.sum(residue**2))
    
    total_energy = sum(energies)
    energy_fracs = [e / total_energy * 100 for e in energies]
    
    # Labels
    labels = [f'IMF {i}' for i in range(len(imfs))] + ['Residue']
    
    # Plot
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.bar(labels, energy_fracs, color='steelblue', alpha=0.7, edgecolor='black')
    
    # Color residue differently
    bars[-1].set_color('darkred')
    
    ax.set_ylabel('Energy Fraction (%)')
    ax.set_title(f'{symbol} - Energy Distribution Across IMFs')
    ax.set_xticklabels(labels, rotation=45)
    
    # Add value labels
    for bar, frac in zip(bars, energy_fracs):
        if frac > 1:
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                   f'{frac:.1f}%', ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.show()
    
    return energy_fracs

# Plot for BTC
_ = plot_energy_distribution('BTC-USD', data['BTC-USD'], decomposer)

## 6. Trend Extraction Comparison

In [ ]:
# Compare trend extraction across assets
fig, axes = plt.subplots(len(data), 1, figsize=(14, 4 * len(data)))

for ax, (sym, prices) in zip(axes, data.items()):
    imfs, residue = decomposer.decompose(prices)
    
    # Normalize for comparison
    prices_norm = (prices - prices.min()) / (prices.max() - prices.min())
    residue_norm = (residue - residue.min()) / (residue.max() - residue.min())
    
    ax.plot(prices_norm, 'b-', alpha=0.5, linewidth=0.8, label='Price (normalized)')
    ax.plot(residue_norm, 'r-', linewidth=2, label='Trend (residue)')
    ax.set_ylabel(sym)
    ax.legend(loc='upper left')
    ax.set_title(f'{sym} - Extracted Trend')

axes[-1].set_xlabel('Sample Index')
plt.tight_layout()
plt.show()

## 7. Reconstruction Verification

In [ ]:
# Verify perfect reconstruction
print("Reconstruction Verification")
print("=" * 50)

for sym, prices in data.items():
    imfs, residue = decomposer.decompose(prices)
    reconstructed = imfs.sum(axis=0) + residue
    
    max_error = np.max(np.abs(prices - reconstructed))
    rel_error = max_error / np.max(np.abs(prices))
    
    print(f"{sym:<12} Max error: {max_error:.2e}  Relative: {rel_error:.2e}")

## 8. Rolling SNR Analysis

In [ ]:
def rolling_snr(prices, window=256, step=64, decomposer=None, noise_imfs=2):
    """Compute rolling SNR over time."""
    if decomposer is None:
        decomposer = ICEEMDAN(mode=ProcessingMode.FINANCE)
    
    snrs = []
    positions = []
    
    for i in range(0, len(prices) - window, step):
        chunk = prices[i:i+window]
        result = compute_snr(chunk, decomposer, noise_imfs)
        snrs.append(result['snr'])
        positions.append(i + window // 2)
    
    return np.array(positions), np.array(snrs)

In [ ]:
# Rolling SNR for BTC
symbol = 'BTC-USD'
prices = data[symbol]

print(f"Computing rolling SNR for {symbol}...")
positions, snrs = rolling_snr(prices, window=200, step=50, decomposer=decomposer)

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Price
axes[0].plot(prices, 'b-', linewidth=0.8)
axes[0].set_ylabel('Price')
axes[0].set_title(f'{symbol} - Price and Rolling SNR')

# Rolling SNR
axes[1].plot(positions, snrs, 'g-', linewidth=1.5)
axes[1].axhline(y=3, color='orange', linestyle='--', label='Tradeable threshold')
axes[1].axhline(y=1, color='red', linestyle='--', label='Noise threshold')
axes[1].fill_between(positions, snrs, alpha=0.3, color='green')
axes[1].set_ylabel('SNR')
axes[1].set_xlabel('Sample Index')
axes[1].legend()
axes[1].set_title('Rolling Signal-to-Noise Ratio (window=200)')

plt.tight_layout()
plt.show()

print(f"\nSNR Statistics:")
print(f"  Mean: {np.mean(snrs):.2f}")
print(f"  Min:  {np.min(snrs):.2f}")
print(f"  Max:  {np.max(snrs):.2f}")
print(f"  % above threshold: {100 * np.mean(snrs > 3):.1f}%")

## 9. Summary

This demo showed:

1. **Basic decomposition** - Extract IMFs from real price data
2. **Denoising** - Remove high-frequency noise
3. **Trend extraction** - Isolate the residue as trend
4. **SNR analysis** - Quantify signal quality
5. **Multi-asset comparison** - Compare tradeability across markets
6. **Rolling analysis** - Track SNR over time

**Key takeaway:** Use SNR to decide if a market is worth trading.
- SNR > 10: Strong signal
- SNR > 3: Tradeable
- SNR < 1: Pure noise, skip